In [1]:
# Definir la ruta de las carpetas
ruta_DE2_train = "/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/train"
ruta_DE2_val = "/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/val"
ruta_DE2_test = "/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test"

In [2]:
# Importar el Generador de datos
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


/opt/miniconda3/envs/mamografia/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
# Creamos el generador para las carpetas "train", "val" y "test"
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [4]:
# Cargar datos en Keras
IMG_SIZE = (224, 224)
BATCH_SIZE = 8

train_gen = train_datagen.flow_from_directory(
    ruta_DE2_train,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    ruta_DE2_val,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_gen = test_datagen.flow_from_directory(
    ruta_DE2_test,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print(train_gen.class_indices)

Found 232 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
{'birads_3': 0, 'birads_4': 1, 'birads_5': 2}


In [5]:
# Calcular Class Weights
# birads_3: 0
# birads_4: 1
# birads_5: 2

import numpy as np
from sklearn.utils.class_weight import compute_class_weight

y_train = train_gen.classes

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print(class_weights)

{0: 0.3925549915397631, 1: 2.8641975308641974, 2: 9.666666666666666}


In [6]:
# Modelo con DenseNet121 (Transfer Learning)
# Construyendo el modelo usando la CNN preentrenada DenseNet121
base_model = DenseNet121(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

2026-01-25 17:26:33.631522: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2026-01-25 17:26:33.631711: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-25 17:26:33.631719: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-25 17:26:33.631950: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-25 17:26:33.631994: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Clasificador
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation="softmax")(x) # 3: es el número de clases

model = Model(inputs=base_model.input, outputs=outputs)

In [8]:
# Compilación (enfoque clínico)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)

In [9]:
# Entrenamiento
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=7,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=4,
        min_lr=1e-6
    ),
    ModelCheckpoint(
        "densenet121_etapa2_birads3_4_5.keras",
        monitor="val_loss",
        save_best_only=True
    )
]

In [10]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    class_weight=class_weights,
    callbacks=callbacks
)

Epoch 1/40


2026-01-25 17:27:23.227464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.4871 - auc: 0.4530 - loss: 3.1152 - precision: 0.4909 - recall: 0.4655 - val_accuracy: 0.3800 - val_auc: 0.6587 - val_loss: 1.0442 - val_precision: 0.5000 - val_recall: 0.2800 - learning_rate: 1.0000e-04
Epoch 2/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 938ms/step - accuracy: 0.2974 - auc: 0.4590 - loss: 2.3896 - precision: 0.3005 - recall: 0.2759 - val_accuracy: 0.0800 - val_auc: 0.6360 - val_loss: 2.1253 - val_precision: 0.0488 - val_recall: 0.0400 - learning_rate: 1.0000e-04
Epoch 3/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 932ms/step - accuracy: 0.3319 - auc: 0.4220 - loss: 2.7599 - precision: 0.3180 - recall: 0.2974 - val_accuracy: 0.2000 - val_auc: 0.6475 - val_loss: 1.6045 - val_precision: 0.1515 - val_recall: 0.1000 - learning_rate: 1.0000e-04
Epoch 4/40
29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 934ms/step - accuracy: 0.3103 - auc: 0.4547 - loss: 2.5288 - precision: 0.3151 - recall: 0.2974 - val_accuracy: 0.3400 - val_auc: 0.6529 - val_loss: 1.3086 - v

In [11]:
# Guardar modelo
model.save("densenet121_etapa2_birads_3-5_gradcam.keras")

In [14]:
y_pred = model.predict(test_gen)

7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step 


In [15]:
y_pred.shape

(50, 3)

In [16]:
y_pred[:5]

array([[0.3054062 , 0.6738747 , 0.02071914],
       [0.8490527 , 0.06433421, 0.08661313],
       [0.6474466 , 0.17456596, 0.17798746],
       [0.29826802, 0.67851573, 0.02321629],
       [0.49525684, 0.43923908, 0.06550407]], dtype=float32)

In [17]:
class_names = list(test_gen.class_indices.keys())
print(class_names)

['birads_3', 'birads_4', 'birads_5']


In [18]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Predicciones
y_pred = model.predict(test_gen, verbose=1)
y_pred_classes = np.argmax(y_pred, axis=1)

# Etiquetas reales
y_true = test_gen.classes

# Reporte
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 520ms/step
              precision    recall  f1-score   support

    birads_3       0.85      0.40      0.54        43
    birads_4       0.12      0.33      0.18         6
    birads_5       0.07      1.00      0.13         1

    accuracy                           0.40        50
   macro avg       0.35      0.58      0.28        50
weighted avg       0.75      0.40      0.49        50

[[17 14 12]
 [ 3  2  1]
 [ 0  0  1]]


In [19]:
import pandas as pd

results = pd.DataFrame({
    "image_path": test_gen.filepaths,
    "y_true": y_true,
    "y_pred": y_pred_classes
})

results["correct"] = results["y_true"] == results["y_pred"]

In [23]:
results.head()

,image_path,y_true,y_pred,correct
0,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_3/BIRADS_3_240.png,0,1,False
1,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_3/BIRADS_3_241.png,0,0,True
2,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_3/BIRADS_3_242.png,0,0,True
3,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_3/BIRADS_3_243.png,0,1,False
4,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_3/BIRADS_3_244.png,0,0,True


In [24]:
birads4_wrong = results[
    (results.y_true == 1) & (results.correct == False)
]

birads4_wrong.sample(1)

,image_path,y_true,y_pred,correct
48,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_4/BIRADS_4_39.png,1,2,False


In [25]:
birads4_correct = results[
    (results.y_true == 1) & (results.correct == True)
]

birads4_correct.sample(1)

,image_path,y_true,y_pred,correct
44,/Users/claracelestechavezcotrina/Downloads/DATA_TESIS/dataset_etapa2/test/birads_4/BIRADS_4_35.png,1,1,True


In [26]:
image_path = birads4_wrong.sample(1).image_path.values[0]

In [ ]:
# 2. Cargo imagen
img, img_array = load_image(image_path)

In [22]:
import pandas as pd

pd.set_option("display.max_colwidth", None)